# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### **Import basic package required for this assignment**

In [50]:
import os
import pandas as pd
import src.constants as C


### **Make sure right sources are getting picked**

In [51]:
from src.base_transformer import BaseDBTransformer


### **Make sure the latest class are getting loaded.**

In [52]:
# 1. Import the modules first (so Python knows them)
import importlib
import src.product_transformer
import src.cart_transformer
import src.purchase_transformer
import src.discount_transformer
import src.constants

# 2. Reload them (useful after edits)
importlib.reload(src.product_transformer)
importlib.reload(src.cart_transformer)
importlib.reload(src.purchase_transformer)
importlib.reload(src.discount_transformer)
importlib.reload(src.constants)

# 3. Import the classes fresh from the reloaded modules
from src.product_transformer import ProductTransformer
from src.cart_transformer import CartTransformer
from src.purchase_transformer import PurchaseTransformer
from src.discount_transformer import DiscountTransformer
import src.constants
import random

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

In [53]:
# import mysql.connector

# # Establish the connection
# connection = mysql.connector.connect(
#     host="localhost",       # e.g., "localhost" or an IP address
    #  user="root",   # e.g., "root"
#     password="root@321", # Your MySQL password
#     database="ORDERS"  # Name of the database to connect to
# )

# # Check if the connection is successful
# if connection.is_connected():
#     print("Connection successful!")

# # Close the connection
# connection.close()


In [54]:
from sqlalchemy import create_engine, URL

url_object = URL.create(
    drivername="mysql+mysqlconnector",
    username="root",
    password="root@321",  # raw password, no need to encode
    host="localhost",
    database="ORDERS"
)

engine = create_engine(url_object)


In [55]:
pt = ProductTransformer(engine)


Initialized(Base) for table:  product  size =  (61, 9)  primary keys =  [Column('PRODUCT_ID', INTEGER(), table=<product>, primary_key=True, nullable=False)] 
 TBM if multiple primary keys. Handling only one PK =  PRODUCT_ID


In [56]:
print( pt.pk_columns )

type( pt.pk_columns )

pt.pk_name


[Column('PRODUCT_ID', INTEGER(), table=<product>, primary_key=True, nullable=False)]


'PRODUCT_ID'

In [60]:
# pt = ProductTransformer(engine)

# Load existing rows into memor
df = pt.read()

# Insert new product
r = pt.create(301, "IBM Thinkpad Laptop", 2050, 5000.00, 100, 150, 300, 20, 1.50)
print(r)
# pt.create(302, "IBM Thinkpad Laptop", 2050, 5000.00, 100, 150, 300, 20, 1.50)


# Update a product
# pt.update(301, **{C.mrp:4999.99} )

# Delete a product
# pt.delete(302)


Trying to add row in product where 301 with {'PRODUCT_ID': 301, 'PRODUCT_DESC': 'IBM Thinkpad Laptop', 'PRODUCT_CLASS_CODE': 2050, 'PRODUCT_PRICE': 5000.0, 'PRODUCT_QUANTITY_AVAIL': 100, 'WEIGHT': 1.5, 'LEN': 150, 'WIDTH': 300, 'HEIGHT': 20}
insert: 
 {'PRODUCT_ID': 0, 'PRODUCT_DESC': 'IBM Thinkpad Laptop', 'PRODUCT_CLASS_CODE': 2050, 'PRODUCT_PRICE': 5000.0, 'PRODUCT_QUANTITY_AVAIL': 100, 'WEIGHT': 1.5, 'LEN': 150, 'WIDTH': 300, 'HEIGHT': 20} result = 
70 : Row =  (70, 'IBM Thinkpad Laptop', 2050, Decimal('5000.00'), 100, 150, 300, 20, Decimal('1.5000')) 70


TypeError: cannot concatenate object of type '<class 'sqlalchemy.engine.row.Row'>'; only Series and DataFrame objs are valid

In [48]:
pt.data

,PRODUCT_ID,PRODUCT_DESC,PRODUCT_CLASS_CODE,PRODUCT_PRICE,PRODUCT_QUANTITY_AVAIL,LEN,WIDTH,HEIGHT,WEIGHT
0,1,Sky LED 102 CM TV,2050,35000.0,30,905,750,700,15.000
1,2,Sams 192 L4 Single-door Refrigerator,2050,28000.0,15,1802,750,750,25.000
2,3,Jocky Speaker Music System HT32,2050,8900.0,19,908,300,300,5.000
3,4,Cricket Set for Boys,2051,4500.0,10,890,300,200,18.000
4,5,Infant Sleepwear Blue,2052,250.0,50,596,300,100,0.250
...,...,...,...,...,...,...,...,...,...
57,58,Sony Xperia U (Black White),3000,16499.0,50,54,112,12,0.110
58,59,Nikon Coolpix L810 Bridge,3000,14987.0,50,111,76,83,0.430
59,60,Samsung Galaxy Tab 2 P3100,3000,19300.0,50,122,194,10,0.345
60,61,IBM Thinkpad Laptop,2050,5000.0,100,150,300,20,1.500


## **Discount Transformer - demonstration**

In [6]:
# Initialize
discount_csv = "data/discounts.csv"
discount_transformer = DiscountTransformer(file_path=discount_csv)

# Create a random data
cid1 = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
cid2 = "CART" + str(random.randint(100, 999))  # Generate a random cart ID

user_id1 = str(random.randint(100, 999))
user_id2 = str(random.randint(100, 999))

discode1 = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))
discode2 = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))

did1 = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))
did2 = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))

dpct1 = random.randint(5, 10)
dpct2 = random.randint(5, 10)


# Test: Write sample data
sample_discounts = pd.DataFrame([
    {C.did: did1, C.usrid: user_id1, C.dcode: discode1, C.dpct: dpct1, C.dst: 0},
    {C.did: did2, C.usrid: user_id2, C.dcode: discode2, C.dpct: dpct2, C.dst: 0}
])

discount_transformer.update_df(sample_discounts, mode="overwrite")
discount_transformer.read()

,DiscountID,UserID,DiscountCode,DiscountPercent,DiscountStatus
0,DR351,574,SAVE3994,9,0
1,FD535,728,SAVE7531,10,0


## **Purchase Transformer - demonstration**

In [7]:
# Initialize
purchase_csv = "data/purchases.csv"
purchase_transformer = PurchaseTransformer(file_path=purchase_csv)

# Test: Write sample data
sample_purchases = pd.DataFrame([
    {C.prcid: cid1+"_"+user_id1, C.cid: cid1, C.tamt: 772.92, C.did: did1},
    {C.prcid: cid2+"_"+user_id2, C.cid: cid2, C.tamt: 972.00, C.did: did2}
])
purchase_transformer.update_df(sample_purchases, mode="overwrite")
purchase_transformer.read()

,PaymentID,CartID,TotalAmount,DiscountID
0,CART684_574,CART684,772.92,DR351
1,CART735_728,CART735,972.00,FD535


## **Cart Transformer - demonstration**

In [8]:
# Initialize
cart_csv = "data/carts.csv"
cart_transformer = CartTransformer(file_path=cart_csv)

# Test: Write sample data
sample_cart = pd.DataFrame([
    {C.cid: cid1 , C.pid: "FD7839", C.qnt: 4, C.usrid: user_id1},
    {C.cid: cid2 , C.pid: "DR7979", C.qnt: 2, C.usrid: user_id2}
])
cart_transformer.update_df(sample_cart, mode="overwrite")

# Test: Read back
print(cart_transformer.read())


    CartID ProductID  Quantity UserID
0  CART684    FD7839         4    574
1  CART735    DR7979         2    728


# **Business logic - Helper functions**
- These functions after matured will go into the respective transformer classes. 

In [19]:
def getRandomPids():
    products = product_transformer.read()

    selected_products = random.sample(products[C.pid].to_list(), k=3)
    print("Selected Products:", selected_products)
    return selected_products

def addToCart(spids, uid, cid):
    cart = cart_transformer.read()
    products = product_transformer.read(spids)

    records = [{C.cid: cid, C.qnt: random.randint(1, 5),C.usrid: uid, C.pid: id} for id in spids]
    cart = pd.DataFrame.from_records(records)
    cart_transformer.create(cart)
    return

def checkout_with_discount(uid, cid, did):
    cart     =  cart_transformer.read([cid])
    ps  =  product_transformer.read(cart[C.pid].to_list())

    # Compute total price for the cart
    total = ps[C.mrp].sum() * cart[C.qnt].sum()
    prcid = uid + "_" + cid  # Purchase ID based on user and cart ID
    print ( "Total price for cart", cid, "is", total)
    # Checkout with discount
    discount  =  discount_transformer.read([did])
    if not discount.empty:
        total -= total * (discount[C.dpct].values[0] / 100)
    else:
        print(f"Discount {did} not found, aborting checkout. total value = ", total)
        return 
    purchase_entry = [{
        C.prcid: prcid,
        C.cid: cid,
        C.tamt: total,
        C.did: did
    }]
    purchase_transformer.create(pd.DataFrame.from_records(purchase_entry))
    discount_transformer.update({C.did: did, C.dst: 1})
    return

def checkout_without_discount(uid, cid, did, discode, dpct):
    cart      =  cart_transformer.read([cid])
    ps  =  product_transformer.read(cart[C.pid].to_list())

    # Compute total price for the cart
    total = ps[C.mrp].sum() * cart[C.qnt].sum()
    prcid = uid + "_" + cid  # Purchase ID based on user and cart ID
    print("Total price for cart", cid, "is", total)
    # Checkout with discount
    purchase_entry = [{
        C.prcid: prcid,
        C.cid: cid,
        C.tamt: total,
        C.did: did
    }]

    purchase_transformer.create(pd.DataFrame.from_records(purchase_entry))

    discount_entry = [{
        C.did: did,
        C.usrid: uid,
        C.dcode: discode,
        C.dpct: dpct,
        C.dst: 0
    }]
    discount_transformer.create(pd.DataFrame.from_records(discount_entry))
    return

# **Admin APIs**

In [10]:
def getPurchases():
    purchases = purchase_transformer.read()
    print(purchases) 

def getDiscounts():
    discounts = discount_transformer.read()
    print( discounts )

getPurchases()
getDiscounts()

     PaymentID   CartID  TotalAmount DiscountID
0  CART684_574  CART684       772.92      DR351
1  CART735_728  CART735       972.00      FD535
  DiscountID UserID DiscountCode  DiscountPercent  DiscountStatus
0      DR351    574     SAVE3994                9               0
1      FD535    728     SAVE7531               10               0


# **Use cases 1 - Buy products**

In [15]:
# Create a random data
cid = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
user_id = str(random.randint(100, 999))
discode = [ "NEW", "SAVE", "FESTIVE"][random.randint(0, 2)] + str(random.randint(1000, 9999))
did = ["FD","NC","DR"][random.randint(0,2)] + str(random.randint(100, 999))
dpct = random.randint(5, 10)

# Business logic using Helper functions
sps = getRandomPids()
print("Selected \n", sps)

addToCart(sps, user_id, cid)
print("Cart created and products added. \n")
cart = cart_transformer.read([cid])
print(cart)

checkout_without_discount(user_id, cid, did, discode, dpct)
print ( purchase_transformer.read([user_id+"_"+cid]) )

discounts = discount_transformer.read([did])
discounts

Selected Products: ['FD7839', 'DR7979', 'DR2699']
Selected 
 ['FD7839', 'DR7979', 'DR2699']
Cart created and products added. 

     CartID ProductID  Quantity UserID
11  CART694    FD7839         4    563
12  CART694    DR7979         5    563
13  CART694    DR2699         3    563
Total price for cart CART694 is 6430.32
     PaymentID   CartID  TotalAmount DiscountID
5  563_CART694  CART694      6430.32      NC989


,DiscountID,UserID,DiscountCode,DiscountPercent,DiscountStatus
5,NC989,563,SAVE8682,10,0


# **Use cases 2 - Buy products with discount**

In [20]:
# Create a random data
cid = "CART" + str(random.randint(100, 999))  # Generate a random cart ID
user_id = str( random.randint(100, 999) )

# Business logic using Helper functions
sps = getRandomPids()
print("Selected \n", sps)

addToCart(sps, user_id, cid)
print("Cart created and products added. \n")
cart = cart_transformer.read([cid])
print(cart)

checkout_with_discount(user_id, cid, did)
print ( purchase_transformer.read([user_id+"_"+cid]) )

discounts = discount_transformer.read([did])
discounts

Selected Products: ['DR7064', 'FD6114', 'FD8358']
Selected 
 ['DR7064', 'FD6114', 'FD8358']
Cart created and products added. 

     CartID ProductID  Quantity UserID
20  CART110    DR7064         1    497
21  CART110    FD6114         4    497
22  CART110    FD8358         3    497
Total price for cart CART110 is 3364.48
     PaymentID   CartID  TotalAmount DiscountID
7  497_CART110  CART110     3028.032      NC989


,DiscountID,UserID,DiscountCode,DiscountPercent,DiscountStatus
5,NC989,563,SAVE8682,10,0
